# Cortex Cube Quickstart Notebook

Cortex Cube is a multi-agent framework that offers native support for Snowflake tools. Instead of requiring users or developers to choose between RAG with Cortex Search or Text2SQL with Cortex Analyst, let the Cube route the work to the appropriate tool based on the user's request.

CortexCube can be configured to work with 3 types of tools:
- Cortex Search Tool: For unstructured data analysis, which requires a standard RAG access pattern.
- Cortex Analyst Tool: For supporting structured data analysis, which requires a Text2SQL access pattern.
- Python Tool: For supporting custom user operations (using 3rd Party API's), which requires calling arbitrary python.

This notebook walks through how to configure and run a system with all 3 types of tools. The demo is designed to illustrate how the agent can answer questions that require a divserse combination of tools (RAG,Text2SQL, Python, or a combination).

Note that Cortex Cube does not configure the underlying Cortex Search or Cortex Analyst services for the user. Those services must be configured before initializing Cortex Cube.

# Cortex Cube Configuration

## Connection Setup

Authenticate with Snowpark + set token as environment variable for use by the agents.

In [2]:
from agent_gateway import Agent
from agent_gateway.tools import CortexSearchTool, CortexAnalystTool, PythonTool
from snowflake.snowpark import Session
import os, time

connection_parameters = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "role": os.getenv("SNOWFLAKE_ROLE"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
}

snowpark = Session.builder.configs(connection_parameters).create()

## Snowflake Tool Configuration

The Cortex Search Tool and the Cortex Analyst Tool need to be configured as follows. Note that a connection object is required for each config. In the case below we're using the same connection object for both because the services are both in the same account/database/schema. Users have the option to pass in different connection objects as needed.

In [3]:
search_config = {
    "service_name": "SEC_SEARCH_SERVICE",
    "service_topic": "Snowflake's business,product offerings,and performance.",
    "data_description": "Snowflake annual reports",
    "retrieval_columns": ["CHUNK"],
    "snowflake_connection": snowpark,
}

analyst_config = {
    "semantic_model": "sp500_semantic_model.yaml",
    "stage": "ANALYST",
    "service_topic": "S&P500 company and stock metrics",
    "data_description": "a table with stock and financial metrics about S&P500 companies ",
    "snowflake_connection": snowpark,
}

## Python Tool Configuration

Configuring a Python Tool for Cortex Cube requires 1) Python Callable 2) Tool Description (what does the tool do) 3) Output Description (what does the tool output). 

In the example below we create a NewsTool object that submits a HTTP request to a 3rd Party News API. The python callable is passed into the Python Tool as `news_api_func`.To use the tool below get your free token by signing up for an account at thenewsapi.com or just create your own python function and pass it into the PythonTool.

In [4]:
import requests, json


class NewsTool:
    def __init__(self, token, limit) -> None:
        self.api_token = token
        self.limit = limit

    def news_search(self, news_query: str) -> str:
        news_request = f"""https://api.thenewsapi.com/v1/news/all?api_token={self.api_token}&search={news_query}&language=en&limit={self.limit}"""
        response = requests.get(news_request)
        json_response = json.loads(response.content)["data"]

        return str(json_response)


python_config = {
    "tool_description": "searches for relevant news based on user query",
    "output_description": "relevant articles",
    "python_func": NewsTool(token=os.getenv("NEWS_API_TOKEN"), limit=3).news_search,
}

## Cortex Cube Config

After the tools have been configured, initialize them and configure CortexCube

In [5]:
annual_reports = CortexSearchTool(**search_config)
sp500 = CortexAnalystTool(**analyst_config)
news_search = PythonTool(**python_config)

snowflake_tools = [annual_reports, sp500, news_search]
agent = Agent(snowflake_connection=snowpark, tools=snowflake_tools)

INFO:CortexCubeLogger:Cortex Search Tool successfully initialized
INFO:CortexCubeLogger:Cortex Analyst Tool successfully initialized
INFO:CortexCubeLogger:Python Tool successfully initialized
INFO:CortexCubeLogger:Cortex Cube successfully initialized


## Unstructured Data Questions

# Cortex Cube Testing

The 3 types of questions below are designed to showcase the breadth of tool use patterns possible with Cortex Cube. 

- The Structured Data Questions use the Cortex Analyst agent. 
- The Unstructured Data Questions use either the Cortex Search agent or the Python (News API) agent.
- The last section includes a question that requires the use of both types of tools.

## Structured Data Questions

In [6]:
agent("What is the market cap of Apple?")

INFO:CortexCubeLogger:running sp500_semantic_model_cortexanalyst task


'The market cap of Apple is $3,019,131,060,224 or approximately $3.02 trillion.'

In [8]:
agent(
    "Which companies are mentioned as competitors in Snowflake's annual report? What is the market cap of each of those competitors"
)

INFO:CortexCubeLogger:running sec_search_service_cortexsearch task
INFO:CortexCubeLogger:running sp500_semantic_model_cortexanalyst task


....replanning...


INFO:CortexCubeLogger:running sp500_semantic_model_cortexanalyst task


"The competitors mentioned in Snowflake's annual report are Amazon Web Services (AWS), Microsoft Azure (Azure), and Google Cloud Platform (GCP). The market cap of their parent companies are as follows:\n- Microsoft Corporation: $3,150,184,448,000\n- Alphabet Inc. (Google): $2,164,350,779,392\n- Amazon.com, Inc.: $1,917,936,336,896"

In [6]:
agent("What is the market cap of Apple?")

INFO:CortexCubeLogger:running sp500_semantic_model_cortexanalyst task


'The market cap of Apple is $3,019,131,060,224 or approximately $3.02 trillion.'

In [15]:
agent("Which company has the bigger EBITDA, Apple or MSFT?")

INFO:CortexCubeLogger:running SP500_cortexanalyst task
INFO:CortexCubeLogger:running SP500_cortexanalyst task


'Apple has a bigger EBITDA than Microsoft.'

# Unstructured Data Questions

In [6]:
agent("How many customers does Snowflake have?")

INFO:CortexCubeLogger:running sp500_semantic_model_cortexanalyst task
 		You are using the client Snowflake, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb


KeyboardInterrupt: 

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-21' coro=<TaskProcessor._run_task() done, defined at /Users/alherrera/Downloads/Cube/cortex-cube/CortexCube/cube/task_processor.py:129> exception=CortexCubeError('Unexpected error during Cortex Cube Tool request: {\'role\': \'analyst\', \'content\': [{\'type\': \'text\', \'text\': "I apologize, but the question \'How many customers does Snowflake have?\' cannot be answered with the given schema. The schema provided does not contain any information about customers or Snowflake."}]}')>
Traceback (most recent call last):
  File "/Users/alherrera/Downloads/Cube/cortex-cube/CortexCube/tools/snowflake_tools.py", line 341, in asearch
    query_response = self._process_message(
                     ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/alherrera/Downloads/Cube/cortex-cube/CortexCube/tools/snowflake_tools.py", line 377, in _process_message
    if "sql" == response[1]["type"]:
                ~~~~~~~~^^^
IndexErro

Caught unhandled exception: Unexpected error during Cortex Cube Tool request: {'role': 'analyst', 'content': [{'type': 'text', 'text': "I apologize, but the question 'How many customers does Snowflake have?' cannot be answered with the given schema. The schema provided does not contain any information about customers or Snowflake."}]}


In [17]:
agent("Give me recent news about Azure's AI investments")

INFO:CortexCubeLogger:running news_search task


"'Recent news about Azure's AI investments includes:\\n- Article: Microsoft: Navigating AI Investments Amid Market Volatility Source: [GuruFocus](https://www.gurufocus.com/news/2499558/microsoft-navigating-ai-investments-amid-market-volatility)\\n- Article: Microsoft: Leading the AI Revolution Source: [GuruFocus](https://www.gurufocus.com/news/2454844/microsoft-leading-the-ai-revolution)\\n- Article: How do the big AI cloud providers stack up at the end of Q2? Bernstein weighs in Source: [Investing.com](https://www.investing.com/news/stock-market-news/how-do-the-big-ai-cloud-providers-stack-up-at-the-end-of-q2-bernstein-weighs-in-432SI-3581725)"

## Unstructured + Structured Data Questions

In [18]:
agent(
    "What are the top 5 companies in the S&P by EBITDA? Are any of them mentioned as competitors in Snowflake's annual report?"
)

INFO:CortexCubeLogger:running SP500_cortexanalyst task
INFO:CortexCubeLogger:running sec_search_service_cortexsearch task


"The top 5 companies in the S&P by EBITDA are Apple Inc., Microsoft Corporation, Alphabet Inc., Berkshire Hathaway Inc. New, and Amazon.com, Inc. None of these companies are mentioned as competitors in Snowflake's annual report."